## CSV files 4: streaming larger-than-memory datasets
By the end of this lecture you will be able to:
- process larger-than-memory datasets from CSVs with streaming

With streaming Polars can process a full query on a larger-than-memory dataset by:
- reading each CSV file in batches
- adapting its standard operations to work on batches instead of the full dataset at once

In [ ]:
import polars as pl

Obviously it doesn't work for me to provide very large datasets with this course. Instead we will do streaming on a small dataset and you can then apply it to your own larger datasets

In [ ]:
csvFile = "../data/nyc_trip_data_1k.csv"

We start with a simple non-streaming query

In [ ]:
(
    pl.scan_csv(csvFile,parse_dates = True)
    .filter(pl.col("passenger_count") > 5)
    .collect()
    .head(3)
)

We make this streaming by passing `streaming = True` to `collect`

In [ ]:
(
    pl.scan_csv(csvFile,parse_dates = True)
    .filter(pl.col("passenger_count") > 5)
    .collect(streaming=True)
    .head(3)
)

Not all operations support streaming - for those that do not Polars uses a non-streaming approach.

## Streaming joins
We can join data from different CSVs with streaming. In this example we join the CITES trade records with the ISO country data on the `Importer` column
> See the lectures on joining `DataFrames` if you have not encountered these datasets yet

In [ ]:
citesCsvFile = "../data/cites_extract.csv"
isoCSVFile = "../data/countries_extract.csv"
(
    pl.scan_csv(citesCsvFile)
    .join(
        pl.scan_csv(isoCSVFile),
        left_on="Importer",
        right_on="alpha-2", 
        how="inner"
    )
    .collect(streaming=True)
)
        


## Output to a file
In the current release Polars needs to output a `DataFrame` from streaming and so the output of the query must fit in memory. 

In a future release Polars will support writing streaming outputs directly to a file. This lecture will be updated when this functionality is released.

## Profiling
We can profile a query when we use streaming. 

> If you have not encountered `profile` see the lecture on Lazy Groupby in the section on Statistics, Counts and Grouping for an introduction

In [ ]:
groupDf, profileDF = (
    pl.scan_csv(csvFile)
    .groupby("passenger_count")
    .agg(
        pl.col("trip_distance").mean()
    )
    .sort("passenger_count")
    .profile(streaming=True,show_plot=True)
)

## Streaming and common subplan elimination
The query above produced the following notification from Polars

```
Cannot combine 'streaming' with 'common_subplan_elimination'. CSE will be turned off.
```
Common subplan elimination is one of the ways that the query optimiser can optimise queries. It arises in queries where the same action is applied to the same `LazyFrame` in different parts of a query.

## Exercises
There are no exercises here as streaming works in a similar way to operations we have met before.

Try streaming on your own data - if you encounter any problems then get in touch with me to see if we can understand the issue.